In [ ]:
import numpy as np
import pandas as pd
import pyBigWig
import matplotlib.pyplot as plt

In [ ]:
rna_bw = pyBigWig.open("rna.bw")
atac_bw = pyBigWig.open("atac.bigWig")

In [ ]:
rna_intervals =rna_bw.intervals('chr19')
atac_intervals = atac_bw.intervals('chr19')

In [ ]:
#atac_intervals
#rna_bw.stats('chr19',477300,477301)[0]

In [ ]:

rna_base=[]; rna_signal=[]
atac_base=[]; atac_signal=[]
for interv in rna_intervals:
    for i in range(interv[0]+1,interv[1]+1):
        rna_base.append(i)
        rna_signal.append(interv[2])
        
for interv in atac_intervals:
    if interv[2]:
        for i in range(interv[0]+1,interv[1]+1):
            atac_base.append(i)
            atac_signal.append(interv[2])


In [ ]:
#atac_base

In [ ]:

plt.figure()
plt.title("Gene qualification of rna-seq-pipeline")
plt.plot(rna_base, rna_signal)
plt.xlim((0,59000000))

plt.show()


In [ ]:

plt.figure()
plt.title("Signal of atac-seq-pipeline")
plt.plot(atac_base, atac_signal)
plt.xlim((0,59000000))
plt.ylim((0,160))

plt.show()


In [ ]:
part = []
for ranges in rna_intervals:
    for i in range(ranges[0]+1,ranges[1]+1):
        part.append(i)

In [ ]:
#part

In [ ]:
sep_all = []
sep = []
for i in range(len(part)-1):
    if part[i]+1 == part[i+1]:
        sep.append((part[i],rna_bw.stats("chr19",part[i]-1,part[i])[0]))
    else:
        sep.append((part[i],rna_bw.stats("chr19",part[i]-1,part[i])[0]))
        sep_all.append(sep)
        sep = []

In [ ]:
sep_all

In [ ]:
"""
Sum = 0
length = 0
for region in rna_intervals:
    Sum += region[2]*(region[1] - region[0])
    length += region[1]-region[0]
mean = Sum/length
mean
"""

In [ ]:
#deleting constant array and low signal array
region2 = []
for reg in sep_all:
    cnt = 0
    qcnt = 0
    for i in range(len(reg)-1):
        if reg[i][1] == reg[i+1][1]:
            cnt += 1
        if reg[i][1] in mx2:
            qcnt += 1
    if cnt != len(reg)-1 and qcnt > 0:
        region2.append(reg)

In [ ]:
len(region2)

In [ ]:
"""
atac_seq_with_rna_all = []
for reg in region2:
    atac_seq_with_rna = []
    for i in range(reg[0][0]-1000,reg[0][0]):
        atac_seq_with_rna.append((i,atac_bw.stats('chr19',i-1,i)[0]))
    atac_seq_with_rna_all.append(atac_seq_with_rna)
"""

In [ ]:
#qc
sep2 = []
sep_all2= []
ct = 0
for interv in atac_intervals:
    #if interv[2]:
    if interv[2] in mx:
        sep2.append(interv)
        #ct += 1
    else:
        if len(sep2) > 0: # and ct > 0:
            sep_all2.append(sep2)
        sep2=[]
    #    ct = 0

In [ ]:
len(sep_all2)

In [ ]:
#len(atac_seq_with_rna_all)

In [ ]:
#used
mean2 = rna_bw.stats('chr19')[0]
mx2 = rna_bw.stats('chr19',type="max", nBins=3) #10
mx2

In [ ]:
peak_all2 = []
for interv in rna_intervals:
    if interv[2] in mx2:
        peak3 = []
        for i in range(interv[0]+1,interv[1]+1):
            peak3.append((i,interv[2]))
        peak_all2.append(peak3)

In [ ]:
len(peak_all2)

In [ ]:
rna_peak_group = []
for region in peak_all2:
    pre_ = [(i,rna_bw.stats('chr19',i-1,i)[0]) for i in range(region[0][0]-500,region[0][0])]
    app_ = [(i,rna_bw.stats('chr19',i-1,i)[0]) for i in range(region[-1][0]+1,region[-1][0]+501)]
    rna_peak_group.append(pre_ + region + app_)

In [ ]:
rna_peak_group

In [ ]:
#used
mean = atac_bw.stats('chr19')[0]
mx = atac_bw.stats('chr19',type="max", nBins=22) #22
mx

In [ ]:
peak_all = []
for interv in atac_intervals:
    if interv[2] in mx:
        peak3 = []
        for i in range(interv[0]+1,interv[1]+1):
            peak3.append((i,interv[2]))
        peak_all.append(peak3)

In [ ]:
len(peak_all)

In [ ]:
atac_peak_group = []
for region in peak_all:
    pre_ = [(i,atac_bw.stats('chr19',i-1,i)[0]) for i in range(region[0][0]-500,region[0][0])]
    app_ = [(i,atac_bw.stats('chr19',i-1,i)[0]) for i in range(region[-1][0]+1,region[-1][0]+501)]
    atac_peak_group.append(pre_ + region + app_)

In [ ]:
atac_peak_group

In [ ]:
#atac_peak_group.reverse()

In [ ]:
#atac_peak_group

In [ ]:
"""
new_atac2 = []
new_rna2 = []
for rna in rna_peak_group:
    new_atac = []
    new_rna = []
    for atac in atac_peak_group:
        if rna[0][0] > atac[-1][0]:
            new_rna.append(rna)
            new_atac.append(atac)
            new_rna2.append(new_rna)
            new_atac2.append(new_atac)
            break
    #if len(new_atac) > 0:
    #    break
"""

In [ ]:
"""
region=[]
for reg in sep_all2:
    reg2=[]
    for interv in reg:
        for base in range(interv[0]+1,interv[1]+1):
            reg2.append((base,interv[2]))
    region.append(reg2)
"""

In [ ]:
atac_qc_sign = pd.DataFrame(atac_peak_group)
rna_qc_sign = pd.DataFrame(rna_peak_group)

In [ ]:
rna_qc_sign.shape

In [ ]:
atac_qc_sign.shape

In [ ]:
atac_qc_sign.to_csv("atac_qc2.tsv",index=False)
rna_qc_sign.to_csv("rna_qc2.tsv",index=False)